In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y_%m_%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y_%m_%d')

print(today_y_m_d)

2022_09_23


In [3]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'operator_login', 'Дата и время звонка': 'call_date', 
                'Результат звонка': 'call_result', 'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 
                'Комменатрий': 'comment', 'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 
                'ФИО врача': 'doctor_fio', 'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'operator_login',
                'Зал №': 'hall_number', 'Логин naumen': 'operator_login', 'МО закрепления': 'mo', 
                'Отсутствует на рабочем месте': 'absence', 'login': 'operator_login', 'procedure_rec_status': 'group_procedure',
                'patient_group': 'group_name'}

In [4]:
# путь папки с данными по звонкам
calls_data_path = os.path.join(Path.home(), 'Documents', 'GitHub', 'final_framework')
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join('C:\\', *calls_data_path)
calls_data_path = os.path.join(calls_data_path, 'calls_data_20220802_today.xlsx')
print(calls_data_path)

C:\Users\VAIO\Documents\GitHub\final_framework\calls_data_20220802_today.xlsx


In [5]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = os.path.join('\\t999\Сетевой диск\pdn\data\patients_current_statuses_path')
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join(r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

\\t999\Сетевой диск\pdn\data\patients_current_statuses_path


In [6]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

In [7]:
%%time
calls_data_reading = pd.read_excel(calls_data_path)
calls_data_reading.shape[0]

CPU times: total: 1min 28s
Wall time: 1min 28s


335407

In [8]:
calls_data_reading = calls_data_reading.rename(columns=column_names)

In [9]:
%%time
calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')
calls_data_reading['call_date'] = calls_data_reading['call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True,
                                                                                                 errors='coerce').date())
calls_data_reading['next_call_date'] = \
calls_data_reading['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True, errors='coerce').date())

<timed exec>:1: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: total: 1min 29s
Wall time: 1min 29s


In [10]:
calls_data_reading.call_date.unique()[-1]

datetime.date(2022, 9, 19)

In [26]:
calls = pd.read_csv('calls_data.csv', encoding='cp1251', sep=';', header=0)

In [27]:
calls

,МО,МУ,МО где создали программу,Emiasid,Имя,Отчество,Контактный телефон,ФИО врача,Специальность врача,Программа ДН,...,Логин оператора,Дата и время звонка,Дата и время окончания звонка,Результат звонка,Разговор состоялся,Дата и время переноса,Комментарий,Предыдущий комментарий,Кол-во попыток дозвона,fid_alg
0,ГБУЗ КДЦ 6 ДЗМ,КДЦ 6 Филиал ГП 155,ГБУЗ КДЦ 6 ДЗМ,31002862,Татьяна,Анатольевна,9167016266,Курбанова Лаият Курбановна,Врач-терапевт участковый,Ишемическая болезнь сердца женщины,...,pomoshnik_pdn23_n2,20.09.2022 10:06:51,20.09.2022 10:08:56,Разговор состоялся,NaN,NaN,NaN,"Перезвонить, ,терапевт уходит в отпуск записат...",NaN,2008840
1,ГБУЗ ДЦ 5 ДЗМ,ДЦ 5 Ф 3 (ГП 179),ГБУЗ ДЦ 5 ДЗМ,16538414,Надежда,Борисовна,9169662124,Садыкова Арзыкан Надырбековна,Врач-терапевт участковый,Артериальная гипертония женщины; Ишемическая б...,...,pomoshnik_pdn183_n2,20.09.2022 10:06:52,20.09.2022 10:08:11,Разговор состоялся,NaN,NaN,NaN,"не в Москве,сентябрь",NaN,2008971
2,ГП 6 ДЗМ,ГП 6 ДЗМ Ф2,ГП 6 ДЗМ,18576200,Людмила,Викторовна,9175250070,Колесникова Татьяна Юрьевна,Врач-терапевт участковый,Артериальная гипертония женщины; Сахарный диаб...,...,pomoshnik_pdn18_n2,20.09.2022 10:06:52,20.09.2022 10:07:27,Разговор состоялся,NaN,NaN,NaN,не в москве. сентябрь.\n89175250070-актуальный...,NaN,2009032
3,ГБУЗ ГП 64 ДЗМ,ГБУЗ ГП 64 ДЗМ ФИЛИАЛ 2,ГБУЗ ГП 64 ДЗМ,5380977368036,Елена,Михайловна,9191002789,Миспохов Дилшод Эсобоевич,Врач-терапевт участковый,Артериальная гипертония женщины,...,pomoshnik_pdn45_n2,20.09.2022 10:06:52,20.09.2022 10:09:47,Разговор состоялся,NaN,NaN,NaN,"перезвонить , терапевт",NaN,2008033
4,"ГБУЗ ГП № 218 ДЗМ""""","ГБУЗ ГП № 218 ДЗМ""""","ГБУЗ ГП № 218 ДЗМ""""",16802797,Владислав,Вячеславович,9859900256,Федорова Ирина Эдуардовна,Врач-терапевт участковый,Артериальная гипертония мужчины,...,pomoshnik_pdn72_n2,20.09.2022 10:06:53,20.09.2022 10:09:21,Перенос звонка на другой день,NaN,20.09.2022 14:08,Перезвонить,NaN,NaN,2008084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27969,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 5,ГБУЗ ГП 23 ДЗМ,30000008233286,Владимир,Юрьевич,9647750800,Либоракина Елена Юрьевна,Врач общей практики (семейный врач),Артериальная гипертония мужчины,...,pomoshnik_pdn6_n2,22.09.2022 17:45:29,22.09.2022 17:48:27,Перенос звонка на другой день,NaN,30.09.2022 10:00,"нет слотов, номер 22399",NaN,NaN,2116866
27970,ГП 6 ДЗМ,ГП 6 ДЗМ Ф3,ГП 6 ДЗМ,30382228,Анастасия,Федоровна,9686691278,Аралова Дарья Викторовна,Врач-терапевт участковый,Артериальная гипертония женщины,...,pomoshnik_pdn173_n2,22.09.2022 17:45:31,22.09.2022 17:46:12,Перенос звонка на другой день,NaN,29.09.2022 10:00,"Перезвонить, не удобно говорить","перезвонить, записать на исследования",NaN,2106395
27971,"ГБУЗ ГП № 134 ДЗМ""""","Филиал № 3 ГБУЗ ГП № 134 ДЗМ""""","ГБУЗ ГП № 134 ДЗМ""""",22672862,Наталья,Викторовна,9161855202,Латышева Мария Ярославовна,Врач общей практики (семейный врач),Артериальная гипертония женщины,...,pomoshnik_pdn7_n2,22.09.2022 17:45:36,22.09.2022 17:48:24,Перенос звонка на другой день,NaN,29.09.2022 10:00,"перезвонить, контроль самозаписи","перезвонить, записана на кпт 6.09",NaN,2105768
27972,ГБУЗ ДКЦ 1 ДЗМ,ДКЦ 1 Ф 3 (ГП 84),ГБУЗ ДКЦ 1 ДЗМ,16234515,Людмила,Ивановна,9167145228,Межидова Аминат Руслановна,Врач общей практики (семейный врач),Артериальная гипертония женщины,...,pomoshnik_pdn39_n2,22.09.2022 17:45:40,22.09.2022 17:45:49,Не дозвонились,NaN,NaN,NaN,"Перезвонить, записать на заключительный прием",NaN,2115541


In [11]:
august_for_september = pd.read_excel('all_august_to_september_emiasid.xlsx')
august_for_september['emiasid'] = august_for_september['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_9472\3882302525.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  august_for_september['emiasid'] = august_for_september['emiasid'].astype(str).str.replace('\.0', '')


In [12]:
monthes = {'august', 'september'}

In [13]:
%%time
for month in monthes:
    
    
    if 'september' in month:
        calls_data = calls_data_reading[calls_data_reading.call_date >= dt.date(2022, 9, 2)].reset_index(drop=True).copy()
    else:
        calls_data = calls_data_reading.copy()
        
    
    emiasid_touches = \
    calls_data.groupby('emiasid').agg({'call_date': 'count'}).rename(columns={'call_date': 'calls_amount'}).reset_index()
    
    last_calls_data = \
    calls_data.sort_values(by='call_date').drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)
    
    last_calls_data = last_calls_data[['emiasid', 'call_date', 'next_call_date', 'call_result', 'talk_result']]
    
    last_calls_data = last_calls_data.rename(
        columns={'call_date': 'last_call_date', 'call_result': 'last_call_result',
                'talk_result': 'last_talk_result'})
    
    last_comments = \
    calls_data[~calls_data.call_result.isin({'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс'})][
        ['emiasid', 'comment', 'call_date']].sort_values(by='call_date') \
    .drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)[['emiasid', 'comment']] \
    .rename(columns={'comment': 'old_comment'})
    
    
    
    last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
    last_calls_data['last_call_date'] = last_calls_data['last_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    last_calls_data['next_call_date'] = \
    last_calls_data['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    
    
    patients_current_statuses_reading = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx'),
        usecols=['emiasid', 'status', 'doctor_fio', 'mo']
     )
    
    
    patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
    
    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
#     patients_current_statuses['phone_number'] = patients_current_statuses['phone_number'] \
#     .apply(lambda x: str(x).replace('.0', '') if not pd.isna(x) else x)
    
    
    patients_current_statuses_merged = patients_current_statuses.merge(emiasid_touches, on=['emiasid'], how='left')
    patients_current_statuses_merged.calls_amount = patients_current_statuses_merged.calls_amount.fillna(0)
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_calls_data, on=['emiasid'], how='left')
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_comments, on=['emiasid'], how='left')
    
#     if 'september' in month:
#         september_patients_current_statuses_merged = patients_current_statuses_merged.copy()
#     elif 'august' in month:
#         august_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    
    
    if 'august' in month:
        patients_current_statuses_merged = patients_current_statuses_merged[patients_current_statuses_merged.emiasid.isin(
        august_for_september.emiasid)]
    
    patients_current_statuses_merged['month'] = month
    
    patients_current_statuses_merged = patients_current_statuses_merged[[
        'emiasid', 'status', 'old_comment', 'next_call_date', 
        'mo', 'doctor_fio', 
        'calls_amount', 'last_call_date',  'last_call_result', 'last_talk_result',  
        'month'        
    ]]
    

    
    patients_current_statuses_merged.to_excel(
        os.path.join(Path.home(), 'Documents', 'GitHub', 'final_framework', 'merged_statuses',
                     f'{month}_patients_current_statuses_with_calls_info_{today_y_m_d}.xlsx'), index=False)

<timed exec>:30: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:44: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:30: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:44: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: total: 1min 14s
Wall time: 1min 15s
